<a href="https://colab.research.google.com/github/LokeshVadlamudi/EmergingTechnologiesML/blob/master/assignment6TFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing libraries

In [2]:
!pip install tensorflow_model_analysis

     |████████████████████████████████| 1.7MB 5.6MB/s 
     |████████████████████████████████| 63.8MB 62kB/s 
     |████████████████████████████████| 2.2MB 55.2MB/s 
     |████████████████████████████████| 788kB 44.0MB/s 
     |████████████████████████████████| 8.9MB 47.0MB/s 
     |████████████████████████████████| 358kB 46.1MB/s 
     |████████████████████████████████| 153kB 46.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 829kB 44.6MB/s 
     |████████████████████████████████| 2.0MB 47.9MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 174kB 4.3MB/s 
     |████████████████████████████████| 184kB 6.1MB/s 
     |████████████████████████████████| 92kB 6.0MB/s 
     |████████████████████████████████| 153kB 6.8MB/s 
     |████████████████████████████████| 174kB 8.0MB/s 
     |████████████████████████████████| 256kB 8.0MB/s 
     |█

In [2]:
!pip install tfx

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 1.5MB 41.0MB/s 
     |████████████████████████████████| 153kB 48.4MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
     |████████████████████████████████| 2.8MB 51.6MB/s 
     |████████████████████████████████| 112kB 48.6MB/s 
     |████████████████████████████████| 1.3MB 54.1MB/s 
     |████████████████████████████████| 389kB 46.4MB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
     |████████████████████████████████| 3.3MB 45.8MB/s 
     |████████████████████████████████| 296kB 47.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=20f8985cc199d6ae59bdef9bb88709993df7ecb7775e371d83b8e9e79c51f566
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from typing import List, Text

import absl
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

In [4]:
_pipeline_name = 'mnist_native_keras'

In [5]:
_mnist_root = os.path.join(os.environ['HOME'], 'mnist')
_data_root = os.path.join(_mnist_root, 'data')

In [6]:
_module_file = os.path.join(_mnist_root, 'mnist_utils_native_keras.py')
_module_file_lite = os.path.join(
    _mnist_root, 'mnist_utils_native_keras_lite.py')

In [7]:
_serving_model_dir = os.path.join(_mnist_root, 'serving_model', _pipeline_name)
_serving_model_dir_lite = os.path.join(
    _mnist_root, 'serving_model_lite', _pipeline_name)

In [8]:
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)

In [9]:
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [10]:
_beam_pipeline_args = [
    '--direct_running_mode=multi_processing',
    # 0 means auto-detect based on on the number of CPUs available
    # during execution time.
    '--direct_num_workers=0',
]


In [11]:
def _create_pipeline(pipeline_name: Text, pipeline_root: Text, data_root: Text,
                     module_file: Text, module_file_lite: Text,
                     serving_model_dir: Text, serving_model_dir_lite: Text,
                     metadata_path: Text,
                     beam_pipeline_args: List[Text]) -> pipeline.Pipeline:
  """Implements the handwritten digit classification example using TFX."""
  examples = external_input(data_root)

  # Brings data into the pipeline.
  example_gen = ImportExampleGen(input=examples)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  # Performs anomaly detection based on statistics and data schema.
  example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])

  # Performs transformations and feature engineering in training and serving.
  transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)

  def _create_trainer(module_file, instance_name):
    return Trainer(
        module_file=module_file,
        custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=5000),
        eval_args=trainer_pb2.EvalArgs(num_steps=100),
        instance_name=instance_name)

  # Uses user-provided Python function that trains a Keras model.
  trainer = _create_trainer(module_file, 'mnist')

  # Trains the same model as the one above, but converts it into a TFLite one.
  trainer_lite = _create_trainer(module_file_lite, 'mnist_lite')

  # TODO(b/150949276): Add resolver back once it supports two trainers.

  # Uses TFMA to compute an evaluation statistics over features of a model and
  # performs quality validation of a candidate model.
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='image_class')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.config.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.8})))
          ])
      ])

  eval_config_lite = tfma.EvalConfig()
  eval_config_lite.CopyFrom(eval_config)
  # Informs the evaluator that the model is a TFLite model.
  eval_config_lite.model_specs[0].model_type = 'tf_lite'

  # Uses TFMA to compute the evaluation statistics over features of a model.
  evaluator = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      eval_config=eval_config,
      instance_name='mnist')

  # Uses TFMA to compute the evaluation statistics over features of a TFLite
  # model.
  evaluator_lite = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer_lite.outputs['model'],
      eval_config=eval_config_lite,
      instance_name='mnist_lite')

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)),
      instance_name='mnist')

  # Checks whether the TFLite model passed the validation steps and pushes the
  # model to a file destination if check passed.
  pusher_lite = Pusher(
      model=trainer_lite.outputs['model'],
      model_blessing=evaluator_lite.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir_lite)),
      instance_name='mnist_lite')

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          transform,
          trainer,
          trainer_lite,
          evaluator,
          evaluator_lite,
          pusher,
          pusher_lite,
      ],
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      beam_pipeline_args=beam_pipeline_args)

In [12]:
if __name__ == '__main__':
  absl.logging.set_verbosity(absl.logging.INFO)
  BeamDagRunner().run(
      _create_pipeline(
          pipeline_name=_pipeline_name,
          pipeline_root=_pipeline_root,
          data_root=_data_root,
          module_file=_module_file,
          module_file_lite=_module_file_lite,
          serving_model_dir=_serving_model_dir,
          serving_model_dir_lite=_serving_model_dir_lite,
          metadata_path=_metadata_path,
          beam_pipeline_args=_beam_pipeline_args))

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.


INFO:absl:Component ImportExampleGen depends on [].
INFO:absl:Component ImportExampleGen is scheduled.
INFO:absl:Component StatisticsGen depends on ['Run[ImportExampleGen]'].
INFO:absl:Component StatisticsGen is scheduled.
INFO:absl:Component SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Component SchemaGen is scheduled.
INFO:absl:Component ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Component ExampleValidator is scheduled.
INFO:absl:Component Transform depends on ['Run[SchemaGen]', 'Run[ImportExampleGen]'].
INFO:absl:Component Transform is scheduled.
INFO:absl:Component Trainer.mnist depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Component Trainer.mnist is scheduled.
INFO:absl:Component Trainer.mnist_lite depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Component Trainer.mnist_lite is scheduled.
INFO:absl:Component Evaluator.mnist depends on ['Run[ImportExampleGen]', 'Run[Trainer.mnist]'].
INFO:absl:Component Evaluator

RuntimeError: ignored